In [1]:
import os
import json
from pathlib import Path
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt


from data import get_dataset, get_zoo_elephants_images_and_labels, get_ELEP_images_and_labels
from train import SiameseModel


import tensorflow as tf
from tensorflow.keras import optimizers

In [2]:
with open('hyperparameters/initial_run.json', 'rb') as f:
    params = json.load(f)

In [3]:
params['batch_size'] = 5
params['val_batch_size'] = 5

In [4]:
data_dir = '/Users/deepakduggirala/Documents/ELPephant-cropped'

In [5]:
cache_files = {
        'train': 'ELP_train.cache',
        'val': 'ELP_val.cache'
    }
train_ds, val_ds, N = get_dataset(get_ELEP_images_and_labels, params, data_dir, cache_files)

In [6]:
siamese_model = SiameseModel(params, True)
siamese_model.compile(optimizer=optimizers.Adam(params['lr']))
input_shape = (None, params['image_size'], params['image_size'], 3)
siamese_model.compute_output_shape(input_shape=input_shape)

TensorShape([None, 256])

In [9]:
siamese_model.load_weights('best_weights/20220422-002754/weights.ckpt')

In [11]:
str(Path('best_weights/20220422-002754/') / 'weights.ckpt')

'best_weights/20220422-002754/weights.ckpt'

In [16]:
RUN_DATETIME_STR = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
RUN_DATETIME_STR

'20220422-002754'

In [17]:
filepath = Path('best_weights') / RUN_DATETIME_STR / 'weights.ckpt'

In [18]:
filepath.parent.mkdir(parents=True, exist_ok=True)

In [19]:
best_cp_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor='val_loss',
    filepath=filepath,
    save_weights_only=True,
    verbose=1,
    mode='min',
    save_best_only=True)

In [25]:
filepath = Path('latest_models') / RUN_DATETIME_STR / 'model.ckpt'
filepath.parent.mkdir(parents=True, exist_ok=True)
latest_cp_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor='val_loss',
    filepath=filepath,
    save_weights_only=False,
    verbose=1)

In [26]:
siamese_model.fit(train_ds.take(1),
                  epochs=1,
                  validation_data=val_ds.take(1),
                  callbacks=[latest_cp_callback, best_cp_callback])

1/1 [==============================] - ETA: 0s - loss: 0.0000e+00
Epoch 00001: saving model to latest_models/20220422-002754/model.ckpt


2022-04-22 00:29:39.583171: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: latest_models/20220422-002754/model.ckpt/assets


/Users/deepakduggirala/miniforge3/envs/dl/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/Users/deepakduggirala/miniforge3/envs/dl/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)



Epoch 00001: val_loss improved from inf to 0.00000, saving model to best_weights/20220422-002754/weights.ckpt
1/1 [==============================] - 12s 12s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
